In [1]:
import pandas as pd
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import numpy as np
import os
from time import time

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#define required function 

#create and store weights
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

#create and store bias
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#build convolutional layer
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

#max pool
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

#randomly choose n samples in the dataset to train
def random_batch(n, imgs, labels):
    index = np.arange(0 , len(imgs))
    np.random.shuffle(index)
    idx = index[0:n]
    imgs_random = [imgs[ i] for i in idx]
    labels_random = [labels[ i] for i in idx]

    return np.asarray(imgs_random), np.asarray(labels_random)



In [3]:
#change to the directory of data 
os.chdir('/Users/rosen/Documents/DS/Machine_Learning_1/project/train')

#loading the images
data=[]
for i in range(1,50001):
    x=str(i)+'.png'
    y=mpimg.imread(x)
    data.append(y)
    
#loading the labels and change the data type from string to array.
label_o=pd.read_csv('trainLabels.csv')
class_le = LabelEncoder()
label_ = class_le.fit_transform(label_o['label'])   
label_new=[]
for i in range(0,len(label_)):
    a=np.array([0,0,0,0,0,0,0,0,0,0])
    a[label_[i]]=1
    label_new.append(a)
label = np.array(label_new)

#split the data into training set and testing set.
X_train=np.array(data[:40000]).reshape(-1,3072)
X_test=np.array(data[40000:]).reshape(-1,3072)
y_train=np.array(label[:40000])
y_test=np.array(label[40000:])

In [4]:
#building input holder
x  = tf.placeholder("float", shape=[None, 3072])
y_ = tf.placeholder("float", shape=[None, 10])
x_image = tf.reshape(x, [-1,32,32,3])

#building two convolutional layers with rectified linear activations and max pool function.
W_conv1 = weight_variable([5, 5, 3, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


#building two fully connected layers
W_fc1 = weight_variable([8 * 8 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool_flat = tf.reshape(h_pool2, [-1, 8 * 8 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

#building traning step to reduce cross entropy.
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#test the accuracy
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


sess = tf.Session()
sess.run(tf.global_variables_initializer())
   
start=time()

#train the model 20000 times, each time ramdonly pick 50 samples as the batch.
for i in range(20000):
    
    batch = random_batch(50, X_train, y_train)
    #show the traning accuracy every 100 steps.
    if i%100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
print ("test accuracy %g"% sess.run(accuracy, feed_dict={x: X_test, y_: y_test, keep_prob: 1.0}) )
end = time()

#print the time cost
cost = str((end-start)/60.)+' minutes'
print('Total time cost: '+cost)

step 0, training accuracy 0.06
step 100, training accuracy 0.26
step 200, training accuracy 0.3
step 300, training accuracy 0.38
step 400, training accuracy 0.5
step 500, training accuracy 0.42
step 600, training accuracy 0.62
step 700, training accuracy 0.48
step 800, training accuracy 0.48
step 900, training accuracy 0.42
step 1000, training accuracy 0.54
step 1100, training accuracy 0.4
step 1200, training accuracy 0.5
step 1300, training accuracy 0.6
step 1400, training accuracy 0.54
step 1500, training accuracy 0.54
step 1600, training accuracy 0.5
step 1700, training accuracy 0.48
step 1800, training accuracy 0.52
step 1900, training accuracy 0.6
step 2000, training accuracy 0.62
step 2100, training accuracy 0.58
step 2200, training accuracy 0.52
step 2300, training accuracy 0.56
step 2400, training accuracy 0.54
step 2500, training accuracy 0.6
step 2600, training accuracy 0.68
step 2700, training accuracy 0.6
step 2800, training accuracy 0.5
step 2900, training accuracy 0.66
st